<img src="http://certificate.tpq.io/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# AI in Finance

**Workshop at Texas State University (October 2023)**

**_Advanced Financial Examples_**

Dr. Yves J. Hilpisch | The Python Quants GmbH | http://tpq.io

## Imports

In [ ]:
import numpy as np
import pandas as pd
from pylab import plt
plt.style.use('seaborn-v0_8')
%config InlineBackend.figure_format = 'svg'

In [ ]:
import warnings
warnings.simplefilter('ignore')

## Black-Scholes-Merton

**Estimation**

This example shows how a Deep Neural Network (DNN) can learn to price options in the BSM (1973) model.

### Option Values

In [ ]:
from itertools import product

In [ ]:
from bsm import bsm_call_value

In [ ]:
bsm_call_value(S0=100, K=105, T=1, r=0.05, sigma=0.2)

In [ ]:
n = 4

In [ ]:
S0_ = np.linspace(80, 120, n)
S0_

In [ ]:
K_ = np.linspace(80, 120, n)
K_

In [ ]:
T_ = np.linspace(0.5, 1.5, n)
T_

In [ ]:
r_ = np.linspace(0.01, 0.05, n)
r_

In [ ]:
sigma_ = np.linspace(0.1, 0.3, n)
sigma_

In [ ]:
list(product(S0_, K_))[:8]

In [ ]:
data = pd.DataFrame()

In [ ]:
%%time
for S0, K, T, r, sigma in product(S0_, K_, T_, r_, sigma_):
    value = bsm_call_value(S0, K, T, r, sigma)
    res = pd.DataFrame({'S0': S0, 'K': K, 'T': T, 'r': r,
                        'sigma': sigma, 'value': value}, index=[0])
    data = pd.concat((data, res), ignore_index=True)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
mu, std = data.mean(), data.std()

In [ ]:
data_ = (data - mu) / std  # that's important here ...

## `sklearn`

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# MLPRegressor?

In [ ]:
model = MLPRegressor(hidden_layer_sizes=[512, 512, 512],
                     max_iter=2000, learning_rate_init=0.001,
                     activation='relu')

In [ ]:
f = list(data.columns[:5])
f

In [ ]:
%time model.fit(data_[f], data['value'])

In [ ]:
data['estimate'] = model.predict(data_[f])

In [ ]:
data.head()

In [ ]:
mean_squared_error(data['value'], data['estimate'])

In [ ]:
data[['value', 'estimate']].plot();

In [ ]:
data[['value', 'estimate']].iloc[-50:].plot();

In [ ]:
(data['estimate'] - data['value']).hist(bins=50);

### `TensorFlow/Keras`

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(f)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')

In [ ]:
%time model.fit(data_[f], data['value'], epochs=1000, verbose=False)

In [ ]:
data['estimate'] = model.predict(data_[f])

In [ ]:
data.head()

In [ ]:
mean_squared_error(data['value'], data['estimate'])

In [ ]:
data[['value', 'estimate']].plot();

In [ ]:
data[['value', 'estimate']].iloc[-50:].plot();

In [ ]:
(data['estimate'] - data['value']).hist(bins=50);

### Generalization

In [ ]:
d = pd.DataFrame(np.array((
            (80.0,	80.0, 0.5, 0.01, 0.1),  # in-sample
            (102.5, 107.5, 0.8, 0.015, 0.25),  # out-of-sample
            (102.5, 107.5, 0.8, 0.0175, 0.15))),  # out-of-sample
                 index=['o1', 'o2', 'o3'])      

In [ ]:
d

In [ ]:
d_ = (d - mu.iloc[:-1].values) / std.iloc[:-1].values

In [ ]:
d_

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
model.predict(d_)

In [ ]:
bsm_call_value(80, 80, 0.5, 0.01, 0.1)

In [ ]:
bsm_call_value(102.5, 107.5, 0.8, 0.015, 0.25)

In [ ]:
bsm_call_value(102.5, 107.5, 0.8, 0.0175, 0.15)

## Credit Scoring

**Classification**

Data from [Kaggle](https://www.kaggle.com/datasets/prasy46/credit-score-prediction/). The following data sets are smaller subsets of the original ones.

### The Data

In [ ]:
url_train = 'https://certificate.tpq.io/kaggle_credit_score_train.csv'

In [ ]:
url_test = 'https://certificate.tpq.io/kaggle_credit_score_test.csv'

In [ ]:
train = pd.read_csv(url_train, index_col=0)

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.y.hist(bins=35);

In [ ]:
r = {}
for i in range(300, 305):
    r[i] = i + 1

In [ ]:
train['y'] = train['y'].replace(r)

In [ ]:
train['y'] -= 301

In [ ]:
list(set(train['y']))[:10]

In [ ]:
train_ = (train - train.mean()) / train.std()

### Decision Trees

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# DTC = DecisionTreeClassifier(max_depth=25)

In [ ]:
# model = AdaBoostClassifier(estimator=DTC, n_estimators=5)

In [ ]:
model = DecisionTreeClassifier(max_depth=35)

In [ ]:
%time model.fit(train_.iloc[:, :-1], train['y']) 

In [ ]:
%time train['p'] = model.predict(train_.iloc[:, :-1])

In [ ]:
accuracy_score(train['y'], train['p'])

In [ ]:
train['p'].value_counts()

In [ ]:
train[['y', 'p']].plot(kind='hist', bins=35, alpha=0.5);

### `TensorFlow/Keras`

In [ ]:
del train['p']

In [ ]:
n_features = len(train.columns) - 1
n_features

In [ ]:
n_labels = len(set(train['y'])) + 1
n_labels

In [ ]:
train_ = (train - train.mean()) / train.std()

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=len(train.columns) - 1))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_labels, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001))

In [ ]:
%%time
model.fit(train_.iloc[:, :-1], train['y'],
          epochs=500, batch_size=128, verbose=False)

In [ ]:
train['p'] = np.argmax(model.predict(train_.iloc[:, :-1]), axis=1)

In [ ]:
accuracy_score(train['y'], train['p'])

In [ ]:
train.head()

In [ ]:
train[['y', 'p']].plot(kind='hist', bins=35, alpha=0.5);

<img src='http://hilpisch.com/tpq_logo.png' width="35%" align="right">

<br><br><a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:team@tpq.io">ai@tpq.io</a>